<h2><font color = '845ec2'> CS 644 - Final Project </font> </h2>
<h3>
<font color='#065535'>Group Members :- <font color = 'red'>Smeet Kathiria, </font> <font color = 'blue'>Krishna Patel, </font> <font color = 'green'>Suchi Khare </font> 
</font> 
</h3>

# Project Part 4 -> Processing and Predicting live tweets 

In [ ]:
import os
import sys


In [ ]:
os.environ["SPARK_HOME"] = "/usr/spark2.4.3"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [ ]:
import warnings
import csv
import time
import re

from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SQLContext, SparkSession
from pyspark.ml import PipelineModel
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql import Row

from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer

<font color='#065535'>Setting up socket stream, socket port,ip, spark context and checkpoint variables so that we can listen to the streaming data coming on port 9878, Note that we run this notebook after running project part 3 notebook which fetches the twitter data and sends it here. </font> 

In [ ]:
checkpoint_path = "/user/smeetp269618/checkpoint"
streaming_socket_ip = "127.0.0.1"
streaming_socket_port = 9878
streaming_time_interval = 2

sc = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()

sqlContext = SQLContext(sc.sparkContext)
  

sparkStreamContext = StreamingContext(sc.sparkContext, streaming_time_interval )

sparkStreamContext.checkpoint(checkpoint_path)

socket_stream = sparkStreamContext.socketTextStream(streaming_socket_ip, streaming_socket_port)

dStream = socket_stream.window(streaming_time_interval)

print("SparkContext Master: " + sc.sparkContext.master)

SparkContext Master: local[*]


In [ ]:
tok = WordPunctTokenizer()

combined_pat = r'|'.join((r'@[A-Za-z0-9_]+', r'https?://[^ ]+'))

negations_catalog = {"isn't":"is not", "aren't":"are not", "wasn't":"was not", "weren't":"were not",
                "haven't":"have not","hasn't":"has not","hadn't":"had not","won't":"will not",
                "wouldn't":"would not", "don't":"do not", "doesn't":"does not","didn't":"did not",
                "can't":"can not","couldn't":"could not","shouldn't":"should not","mightn't":"might not",
                "mustn't":"must not"}

neg_regex = re.compile(r'\b(' + '|'.join(negations_catalog.keys()) + r')\b')

def tweets_transformer(text):
    soup = BeautifulSoup(text, 'lxml')
    get_soup_text = soup.get_text()
    
    try:
        bom_eliminator = get_soup_text.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        bom_eliminator = get_soup_text
        
    strip_text = re.sub(combined_pat, '', bom_eliminator)
    strip_text = re.sub(r'www.[^ ]+', '', strip_text)
    lower_case = strip_text.lower()
    neg_transform = neg_regex.sub(lambda x: negations_catalog[x.group()], lower_case)
    letters_filter = re.sub("[^a-zA-Z]", " ", neg_transform)
    words = [x for x  in tok.tokenize(letters_filter) if len(x) > 1]
    cleaned_text = (" ".join(words)).strip()
    return cleaned_text

<font color='#065535'>In the below function we process each tweet coming from RDD, we sent that to the tweet to the tweet transformer function after that we have the cleaned tweet, after that we load the trained model and than we convert that tweet into data frame and do prediction on that tweet. Once we get the prediction we print the tweet along with prediction and also store the tweet as csv file inside hadoop at particular location.  </font> 

In [ ]:
msg_dict = {}

def predict_Streaming_Tweets(rdd):

    try:
        sc = SparkSession.builder.appName("twitter").getOrCreate()
        
        trained_model = PipelineModel.load("twitter_sentiment_model")
        tweet = rdd.collect()
        
        if len(tweet) != 0:
            tweet = list(tweet[0])
        else:
            tweet = []

        rows = []
        
        for i in range(len(tweet)):
            transformed_tweet = tweets_transformer(tweet[i])
            rows.append(Row(text=transformed_tweet,original=tweet[i],target=0))

        if len(rows) == 0:
            rows.append(Row(text="empty",target=0))
        
        df = sc.createDataFrame(rows)
        
        pred_Tweets = trained_model.transform(df).collect()

        
        for tweet in pred_Tweets:
            msg_dict["text"] = tweet.text
            msg_dict["sentiment"] = tweet.prediction
            if msg_dict["text"] == 'empty':
                pass
            else:
                print('starting tweet prediction')
                print(msg_dict)
                predicted_tweets = sc.createDataFrame(pred_Tweets)
                predicted_tweets = predicted_tweets.select("text",'prediction').collect()
                predicted_tweets = sc.createDataFrame(predicted_tweets)
                predicted_tweets.write.save(path='/user/smeetp269618/pyspark/tweets', format='csv', mode='append', sep='\t')
                print('prediction and saving is done')
            
    except Exception as e:
        print(e)
        
        


<font color='#065535'> Here we are using dstream to transform the input stream by passing each rdd containing tweet inside the predict_streaming_tweets function which processes and predicts on the tweet text and store it into csv file </font> 

In [ ]:
dStream = dStream.map(lambda x: x.lower()).map(lambda x: x.replace(" rt " , " ")).map(lambda x: x.replace("\n" , " "))
dStream = dStream.reduce(lambda x,y : x + y).map(lambda x: x.split(" $$$$$$ "))
dStream.foreachRDD(lambda rdd : predict_Streaming_Tweets(rdd))


<font color='#065535'>Starting the streaming context and we can see the tweet and its corresponding prediction given by the model </font> 

In [ ]:
sparkStreamContext.start()


starting tweet prediction
{'text': 'rt anna hazare biggest blunder delhi deadliest curse the guy is more deadlier than corona or saars removekejrivirus enda tribute to all who helped in this pandemic by risking their lives even apart we are together in this thank motivate them by sharing this song mehenat dilsethankyou corona coronawarriors doctors healthworkers coronavairus covid raprogi mehenat warriors removekejrivirusdeadly than corona let cooperate with the corona frontliners and help them to serve their duty in an efficient manner', 'sentiment': 1.0}
prediction and saving is done
starting tweet prediction
{'text': 'together we are stronge endrt it is the easiest thing to comment and the most difficult to face when the whole world is going through its most diff endrt it is the easiest thing to comment and the most difficult to face when the whole world is going through its most diff endrt all people in the country would be happy to know that the health condition of former pm dr ma

In [ ]:
sparkStreamContext.awaitTerminationOrTimeout(10)


In [ ]:
sparkStreamContext.stop()

<font color='#065535'>Once we have enough tweets we load all the saved csv files inside local folder </font> 

In [ ]:
!hadoop fs -copyToLocal  pyspark/* /home/smeetp269618/predtweetsfolder


<font color='#065535'>After saving all the csv files inside local we delete it from hadoop to save space </font> 

In [ ]:
!hadoop fs -rm -R  pyspark/tweets

21/04/25 02:09:18 INFO fs.TrashPolicyDefault: Moved: 'hdfs://cxln1.c.thelab-240901.internal:8020/user/smeetp269618/pyspark/tweets' to trash at: hdfs://cxln1.c.thelab-240901.internal:8020/user/smeetp269618/.Trash/Current/user/smeetp269618/pyspark/tweets1619316558219


<font color='#065535'>Below fucntions removes all the empty csv files that we get from hadoop </font> 

In [ ]:
import os, glob
import pandas as pd

path = "/home/smeetp269618/predtweetsfolder/"

all_files = glob.glob(os.path.join(path, "*.csv"))


for f in all_files:
    
    file = open(f)
    file_content = file.read()
    file.close()
    if file_content == "":
        os.remove(f)


<font color='#065535'>Below code combines all the csv files into one csv and store it into dataframe   </font> 

In [ ]:
non_empty_files = glob.glob(os.path.join(path, "*.csv"))

predicted_tweets_df = pd.DataFrame(columns = ['text', 'sentiment'])

for f in non_empty_files:
    
    df = pd.read_csv(f)
    predicted_tweets_df = predicted_tweets_df.append({'text': df.columns[0].split("\t")[0] , 'sentiment' : df.columns[0].split("\t")[1] } , ignore_index=True )

        


<font color='#065535'>Here we can see all the streamed live tweets and its corresponding prediction given by the model </font> 

In [ ]:
predicted_tweets_df

,text,sentiment
0,end is there way you could show news besides r...,0.0
1,endrt he can cure cancer hiv and everything el...,0.0
2,failedpmmodi hear you been in similar boat for...,1.0
3,ssri endrt gujarat govt under the decisive sen...,0.0
4,at kims hospital endwhy corona surround us end...,0.0
...,...,...
309,jsrt endboys be like ll take your breath away ...,1.0
310,uae singapore sending oxygen tankers to ind en...,0.0
311,together we are stronge endrt these environmen...,0.0
312,what do end this covid virus is not simple one...,0.0


<font color='#065535'> We save the tweets and predicitons to csv file which will be used to store it into database</font> 

In [ ]:
predicted_tweets_df.to_csv('/home/smeetp269618/predictedtweets_bigdataproject.csv')